In [ ]:
from flask import Flask, request, url_for, redirect, render_template, jsonify, flash
import pickle
import numpy as np
import os
import pandas as pd
from flask import send_from_directory
import upsilon
import math
import matplotlib.pyplot as plt
from io import BytesIO
import traceback
import base64



# Initialize the Flask app
app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))


# Initialize front end
@app.route('/')
def home():
    return render_template('22-23_HSR_App.html')


#create the light curve plot
def create_plot(dates, mags, period):
    #first, phase-fold the light curve
    phases = dates % period  # Compute the phase values by taking the modulo of time with the period

    #fold to the highest magnitude (minimum brightness) for better readability
    max_index = np.argmax(mags)
    #sort the phases after the phase shift
    phase_shift = phases[max_index]
    phases = (phases + phase_shift)%period

    # Sort the phase values
    sorted_indices = np.argsort(phases)
    phase_folded_dates = phases[sorted_indices]
    phase_folded_mags = mags[sorted_indices]

    # Use '-o' to connect the points with lines and plot markers at each data point
    plt.plot(phase_folded_dates, phase_folded_mags, '-o')  
    plt.xlabel('Time (days)')
    plt.ylabel('Apparent Magnitude')
    plt.title('Light Curve of Your Variable Star')
    # Reverse the y-axis as magnitude is inverse
    plt.gca().invert_yaxis()
    #plt.show()
    
    # Save the plot to a BytesIO buffer
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    plt.close()
    return buffer
  

    
# Initialize back end 
@app.route('/', methods=['GET', 'POST'])
def handle_request():
    if request.method == 'POST':
        try: 
            #get file from button
            file = request.files['file']
            file.save(os.path.join("uploads", file.filename))
            df = pd.read_csv('/Users/lukehuang/22-23_HSR/uploads/' + file.filename, sep='\s+', header=None, encoding='utf-8')
        
            #get data from file
            date = np.array(df[df.columns[0]])
            mag = np.array(df[df.columns[1]])
            err = np.array(df[df.columns[2]])
        
            #get avg magnitude for distance calculations
            avg_apparent_mag = np.average(mag)
        
            #run feature calculation
            e_features = upsilon.ExtractFeatures(date, mag, err)
            e_features.run()
            features = e_features.get_features()
            features_as_list = list(features.values()) 
        
            #delete faulty features
            del features_as_list[8]
            cols = ["amplitude", "cusum", "eta", "hl_amp_ratio", "kurtosis", 
                    "n_points", "period", "period_SNR", "period_uncertainty", 
                    "phase_cusum", "phase_eta", "phi21", "phi31", "quartile31", 
                    "r21", "r31", "shapiro_w", "skewness", "slope_per10", 
                    "slope_per90", "stetson_k", "weighted_mean", "weighted_std"]
        
            #get period for distance calculations
            period = features_as_list[6]
        
            #predict class of star
            df_features = pd.DataFrame([features_as_list], columns = cols)
            pred = model.predict(df_features)-1
            classes = ["classical cepheid", "anomalous cepheid", "type 2 cepheid", 
                       "rr lyrae-type", "mira-type", "heartbeat star", 
                       "delta scuti-type", "eclipsing binary", "non variable"]
            pred_class = classes[pred[0]]

            #get distance calculations
            distance = 0
            abs_mag = 0
            isExtrinsic = False;
            isNonVar = False;
            #classical cepheid
            link = ""
            if pred_class == "classical cepheid":
                abs_mag = -2.43*(math.log(period, 10) - 1) - 4.05
                link = "http://ogle.astrouw.edu.pl/atlas/classical_Cepheids.html"
            #type 2 cepheid
            elif pred_class == "type 2 cepheid":
                abs_mag = -2.521*(math.log(period, 10) - 1.2) + 14.339
                link = "https://ogle.astrouw.edu.pl/atlas/type_II_Cepheids.html"
            #rr lyrae
            elif pred_class == "rr lyrae":
                abs_mag = 0.75
                link = "http://ogle.astrouw.edu.pl/atlas/RR_Lyr.html"
            #mira
            elif(pred_class == "mira"):
                abs_mag = -3.51*(math.log(period, 10) - 3.28) - 7.15
                link = "http://ogle.astrouw.edu.pl/atlas/Miras.html"
            #heartbeat
            elif(pred_class == "heartbeat star"):
                isExtrinsic = True
                link = "https://ogle.astrouw.edu.pl/cont/4_main/var/ogleiv/heartbeat_stars/"
            #delta scuti
            elif(pred_class == "delta scuti"):
                abs_mag = -2.94*(math.log(period, 10)) - 1.34
                link = "http://ogle.astrouw.edu.pl/atlas/delta_Sct.html"
            #eclipsing binary
            elif(pred_class == "eclipsing binary"):
                isExtrinsic = True
                link = "https://www.aavso.org/introduction-why-are-eclipsing-binary-stars-important"
            #nonvariable    
            elif(pred_class == "non variable"):
                isNonVar = True
                link = "https://www.aavso.org/tags/non-variable-stars"
            distance = round(10**((avg_apparent_mag-abs_mag+5)/5))
        
            dist_message = ""
            #extrinsic
            if(not isNonVar and isExtrinsic):
                dist_message = "the period of the star is: " + (str)(period) + " days."
            #intrinsic
            elif(not isNonVar):
                dist_message = "the distance to the star is: "+(str)(distance)+" parsecs."
                    
            #initialize the light curve plot
            plot_buffer = create_plot(date, mag, period)
            plot_image = base64.b64encode(plot_buffer.getvalue()).decode('utf-8')
            
        except Exception as e:
            traceback.print_exc()
            # Return a custom error message instead of an Internal Server Error
            error_message = "Error: Failed to process the uploaded file. Please make sure the data format is correct."
            return render_template('22-23_HSR_App.html', error_message=error_message)        
    
    #show distance/hyperlink/star class on submission
    show = True
    
    return render_template('22-23_HSR_App.html', 
                           show=show,
                           star_class="Your star is a: " + pred_class, 
                           star_distance=dist_message,
                           star_link=link,
                           plot_image=plot_image)



if __name__ == '__main__':
    app.run(debug=False)
 
 


-------------------------------
* WARNING: No pyFFTW detected *
-------------------------------
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jul/2023 14:46:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2023 14:46:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2023 14:46:40] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2023 15:23:30] "POST / HTTP/1.1" 200 -
